# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 18 2023 12:38PM,261338,10,HH-41399,Hush Hush,Released
1,Apr 18 2023 12:38PM,261338,10,HH-41400,Hush Hush,Released
2,Apr 18 2023 12:38PM,261338,10,HH-41401,Hush Hush,Released
3,Apr 18 2023 12:38PM,261338,10,HH-41402,Hush Hush,Released
4,Apr 18 2023 12:26PM,261337,19,ELI04192023,Eli Lilly and Company,Released
5,Apr 18 2023 12:22PM,261336,10,Enova-11676,Emerginnova,Released
6,Apr 18 2023 12:16PM,261335,10,SO000342,Alliance Pharma LTD,Released
7,Apr 18 2023 12:13PM,261334,19,60023062,"GCH Granules USA, Inc.",Released
8,Apr 18 2023 12:13PM,261334,19,60023063,"GCH Granules USA, Inc.",Released
9,Apr 18 2023 12:13PM,261334,19,60023064,"GCH Granules USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
40,261334,Released,27
41,261335,Released,1
42,261336,Released,1
43,261337,Released,1
44,261338,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
261334,NaN,NaN,27.0
261335,NaN,NaN,1.0
261336,NaN,NaN,1.0
261337,NaN,NaN,1.0
261338,NaN,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
260872,16.0,25.0,12.0
261225,3.0,24.0,3.0
261231,9.0,2.0,1.0
261234,0.0,0.0,1.0
261238,4.0,11.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
260872,16,25,12
261225,3,24,3
261231,9,2,1
261234,0,0,1
261238,4,11,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,260872,16,25,12
1,261225,3,24,3
2,261231,9,2,1
3,261234,0,0,1
4,261238,4,11,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,260872,16,25,12
1,261225,3,24,3
2,261231,9,2,1
3,261234,,,1
4,261238,4,11,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 18 2023 12:38PM,261338,10,Hush Hush
4,Apr 18 2023 12:26PM,261337,19,Eli Lilly and Company
5,Apr 18 2023 12:22PM,261336,10,Emerginnova
6,Apr 18 2023 12:16PM,261335,10,Alliance Pharma LTD
7,Apr 18 2023 12:13PM,261334,19,"GCH Granules USA, Inc."
34,Apr 18 2023 12:11PM,261333,10,Eye Pharma Inc
35,Apr 18 2023 12:05PM,261331,10,"Uniderm USA, Inc - Collagenil Line"
36,Apr 18 2023 11:57AM,261329,19,"GUSA Granules USA, Inc."
37,Apr 18 2023 11:56AM,261330,10,"Senseonics, Incorporated"
39,Apr 18 2023 11:41AM,261327,19,MedStone Pharma LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Apr 18 2023 12:38PM,261338,10,Hush Hush,,,4
1,Apr 18 2023 12:26PM,261337,19,Eli Lilly and Company,,,1
2,Apr 18 2023 12:22PM,261336,10,Emerginnova,,,1
3,Apr 18 2023 12:16PM,261335,10,Alliance Pharma LTD,,,1
4,Apr 18 2023 12:13PM,261334,19,"GCH Granules USA, Inc.",,,27
5,Apr 18 2023 12:11PM,261333,10,Eye Pharma Inc,,,1
6,Apr 18 2023 12:05PM,261331,10,"Uniderm USA, Inc - Collagenil Line",,,1
7,Apr 18 2023 11:57AM,261329,19,"GUSA Granules USA, Inc.",,,1
8,Apr 18 2023 11:56AM,261330,10,"Senseonics, Incorporated",,,2
9,Apr 18 2023 11:41AM,261327,19,MedStone Pharma LLC,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 12:38PM,261338,10,Hush Hush,4,,
1,Apr 18 2023 12:26PM,261337,19,Eli Lilly and Company,1,,
2,Apr 18 2023 12:22PM,261336,10,Emerginnova,1,,
3,Apr 18 2023 12:16PM,261335,10,Alliance Pharma LTD,1,,
4,Apr 18 2023 12:13PM,261334,19,"GCH Granules USA, Inc.",27,,
5,Apr 18 2023 12:11PM,261333,10,Eye Pharma Inc,1,,
6,Apr 18 2023 12:05PM,261331,10,"Uniderm USA, Inc - Collagenil Line",1,,
7,Apr 18 2023 11:57AM,261329,19,"GUSA Granules USA, Inc.",1,,
8,Apr 18 2023 11:56AM,261330,10,"Senseonics, Incorporated",2,,
9,Apr 18 2023 11:41AM,261327,19,MedStone Pharma LLC,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 12:38PM,261338,10,Hush Hush,4,,
1,Apr 18 2023 12:26PM,261337,19,Eli Lilly and Company,1,,
2,Apr 18 2023 12:22PM,261336,10,Emerginnova,1,,
3,Apr 18 2023 12:16PM,261335,10,Alliance Pharma LTD,1,,
4,Apr 18 2023 12:13PM,261334,19,"GCH Granules USA, Inc.",27,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 12:38PM,261338,10,Hush Hush,4.0,NaN,NaN
1,Apr 18 2023 12:26PM,261337,19,Eli Lilly and Company,1.0,NaN,NaN
2,Apr 18 2023 12:22PM,261336,10,Emerginnova,1.0,NaN,NaN
3,Apr 18 2023 12:16PM,261335,10,Alliance Pharma LTD,1.0,NaN,NaN
4,Apr 18 2023 12:13PM,261334,19,"GCH Granules USA, Inc.",27.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 12:38PM,261338,10,Hush Hush,4.0,0.0,0.0
1,Apr 18 2023 12:26PM,261337,19,Eli Lilly and Company,1.0,0.0,0.0
2,Apr 18 2023 12:22PM,261336,10,Emerginnova,1.0,0.0,0.0
3,Apr 18 2023 12:16PM,261335,10,Alliance Pharma LTD,1.0,0.0,0.0
4,Apr 18 2023 12:13PM,261334,19,"GCH Granules USA, Inc.",27.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3919733,52.0,10.0,0.0
15,261225,3.0,24.0,3.0
19,3135348,45.0,22.0,13.0
20,1044735,17.0,25.0,16.0
21,522639,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3919733,52.0,10.0,0.0
1,15,261225,3.0,24.0,3.0
2,19,3135348,45.0,22.0,13.0
3,20,1044735,17.0,25.0,16.0
4,21,522639,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,52.0,10.0,0.0
1,15,3.0,24.0,3.0
2,19,45.0,22.0,13.0
3,20,17.0,25.0,16.0
4,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,52.0
1,15,Released,3.0
2,19,Released,45.0
3,20,Released,17.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21
Status,,,,,
Completed,0.0,3.0,13.0,16.0,0.0
Executing,10.0,24.0,22.0,25.0,1.0
Released,52.0,3.0,45.0,17.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21
0,Completed,0.0,3.0,13.0,16.0,0.0
1,Executing,10.0,24.0,22.0,25.0,1.0
2,Released,52.0,3.0,45.0,17.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21
0,Completed,0.0,3.0,13.0,16.0,0.0
1,Executing,10.0,24.0,22.0,25.0,1.0
2,Released,52.0,3.0,45.0,17.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()